In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.spatial import distance
from tqdm import tqdm

import spektral
import matplotlib.pyplot as plt

from spektral.layers.convolutional import GraphSageConv, GCNConv
from spektral.layers.pooling import GlobalAttentionPool
from spektral import utils

import tensorflow as tf
# Set CPU as available physical device
#my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
#tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')
#tf.config.set_visible_devices([], 'CPU')

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Flatten, Concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import mean_squared_error
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.optimizers import Adam

In [2]:
#source = '/media/storage_3/abiricz/Mobilcell/TimeIntervalGraphs/'
source = '/media/Data_storage/Mobilcell/TimeIntervalGraphs_spektral/'
source_pol = '/media/Data_storage/Mobilcell/DayPolygonData/'
files = np.array( sorted([ i for i in os.listdir(source) if 'line' in i ]) )
files[:10]

array(['graphNN_linegraph_samples_series-3_target-1_20181203.npz'],
      dtype='<U56')

In [3]:
dates = np.unique( [ j.split('_')[5][:-4] for j in files ] )
dates

array(['20181203'], dtype='<U8')

In [4]:
tower_info = pd.read_csv( source_pol+'fixed_merged-40_tower_locations.csv' ) ## CHANGED
sort_idx = np.argsort( tower_info.tower_id.values )
tower_info = tower_info.iloc[ sort_idx ]
tower_info.reset_index(inplace=True)

coords = np.unique( tower_info.iloc[:,2:], axis=0 )[:,1:]
coords.shape

(8622, 2)

#### Adjacency matrix as decaying distance matrix

### Ideas:
- node prediction: 
    - use node features
    
    
- link prediction the same way using line graph

- try spektral edge conditioned conv / graph conv to predict node features 
    - stack 3 consequtive steps as node and edge feature sets

### Data loader 

In [5]:
daynum = 0
loaded = np.load( source+'graphNN_linegraph_samples_series-3_target-1_'+dates[daynum]+'.npz' )
print( list( loaded.keys() ) )

nfs = loaded['nf']
ef_mats = loaded['ef_mat']
nfs_line = loaded['nf_line']
nf_line_targets = nfs_line[:,:,-1]
nfs_line = nfs_line[:,:,:-1]
adj_mat_line = loaded['adj_mat_line']

#nf_targets = loaded['nf_target']
#node_filt = loaded['node_filt']
#adj_mat_filt = loaded['adj_mat_filt']
#adj_mat = adj_mat[ np.ix_(node_filt, node_filt) ]
#adj_mat = np.repeat( np.expand_dims(adj_mat, 0), 21, 0 )

['node_filt', 'adj_mat_filt', 'adj_mat_line', 'nf', 'ef_mat', 'nf_line']


In [6]:
adj_mat_line.shape, nfs_line.shape, nf_line_targets.shape, nfs.shape, ef_mats.shape, 

((3150, 3150), (20, 3150, 3), (20, 3150), (20, 3436, 4), (20, 3436, 3436, 4))

In [7]:
nf_line_targets[0][:10]

array([ 2,  1,  0,  0,  0,  0, 11,  0,  0,  2], dtype=uint32)

In [8]:
nfs_line[1,:,-1][:10]

array([ 2,  1,  0,  0,  0,  0, 11,  0,  0,  2], dtype=uint32)

### ML

In [41]:
# Parameters
N = nfs_line.shape[-2]          # Number of nodes in the graphs
F = nfs_line.shape[-1]          # Node features dimensionality
#N_line = nf_train_line.shape[-2]          # Number of nodes in the graphs
#F_line = nf_train_line.shape[-1]          # Node features dimensionality
# Dimensionalities of the targets
n_out = nf_line_targets.shape[-2]
learning_rate = 1e-3      # Learning rate for SGD
epochs = 10              # Number of training epochs
batch_size = 1         # Batch size
es_patience = 5           # Patience fot early stopping

In [ ]:
# Model definition
nf_in = Input(shape=(N, F))
adj_in = Input(shape=(N, N))

gc0 = GCNConv( 32, activation='relu')([nf_in, adj_in])
gc0 = GCNConv( 32, activation='relu')([gc0, adj_in])
gc0 = GCNConv( 1, activation='relu')([gc0, adj_in])

outp = Flatten()(gc0)
#outp = GlobalAttentionPool()(gc0)

In [ ]:
# Build model
model = Model( inputs=[nf_in, adj_in], outputs=outp )
#model = multi_gpu_model(model, 2)
optimizer = Adam( lr=learning_rate, amsgrad=True, decay=1e-6 )
model.compile( optimizer=optimizer, loss='mae' )
model.summary()

In [ ]:
# Train model
# [nf_in, adj_in, ef_in, dm_in, cm_in]
history = model.fit([nfs, adj_mat],
          nf_targets,
          batch_size=batch_size,
          validation_split=0.1,
          epochs=epochs)